In [25]:
import numpy as np

In [26]:
# Initial State
init = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 0, 8]
])
# Required Goal State
final = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

In [27]:
# State class to store info about a state
class State:
    def __init__(self, state, x, y, depth, parent = None):
        self.state = state
        self.x = x
        self.y = y
        self.depth = depth
        self.parent = parent

In [28]:
# Finding 0 in start state
start_x, start_y = np.where(np.array(init) == 0)
start = State(init, start_x[0], start_y[0], 0)

In [29]:
# Helper Functions for swapping in states
def swap_up(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x - 1][y] = state[x - 1][y], state[x][y]
    x = x - 1
    return State(state, x, y, node.depth + 1, node)

def swap_down(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x + 1][y] = state[x + 1][y], state[x][y]
    x = x + 1
    return State(state, x, y, node.depth + 1, node)

def swap_left(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x][y - 1] = state[x][y - 1], state[x][y]
    y = y - 1
    return State(state, x, y, node.depth + 1, node)

def swap_right(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x][y + 1] = state[x][y + 1], state[x][y]
    y = y + 1
    return State(state, x, y, node.depth + 1, node)

In [30]:
# Helper Function for finding visited states
def find_in_open_closed(state, open, closed):
    for i in open:
        if (i.state == state).all():
            return True

    for i in closed:
        if (i.state == state).all():
            return True

    return False

In [31]:
# Processing a state including finding its neighbours and adding them to open
def find_neighbours(node, open, closed, pos):
    if node.x < 2:
        new_node = swap_down(node)
        if (new_node.state == final).all():
            return new_node
        if not find_in_open_closed(new_node.state, open, closed):
            open.insert(pos, new_node)

    if node.x > 0:
        new_node = swap_up(node)
        if (new_node.state == final).all():
            return new_node
        if not find_in_open_closed(new_node.state, open, closed):
            open.insert(pos, new_node)

    if node.y < 2:
        new_node = swap_right(node)
        if (new_node.state == final).all():
            return new_node
        if not find_in_open_closed(new_node.state, open, closed):
            open.insert(pos, new_node)

    if node.y > 0:
        new_node = swap_left(node)
        if (new_node.state == final).all():
            return new_node
        if not find_in_open_closed(new_node.state, open, closed):
            open.insert(pos, new_node)

    return False

In [32]:
# Helper Function to print path from start to goal;
def print_path(goal):
    path = [goal]
    while not (path[0].state == init).all():
        path.insert(0, path[0].parent)
    for i in path:
        print(i.state, end = "\n\n")

In [33]:
# State Space algorithm for bfs or dfs
def state_space_search(algo = "bfs", max_depth = 10):
    open = [start]
    closed = []
    pos = -1 if algo == "bfs" else 0 # Inserting at 0 to use open as a stack or at -1 (last) to use as a queue
    found_goal = False

    max_mem = 0 # save maximum memory usage

    while len(open):
        max_mem = max(max_mem, len(open) + len(closed))
        n = open.pop(0)

        if algo == "dfs" and n.depth > max_depth: # keeping a max depth for avoiding TLE
            continue

        closed.append(n)
        found_goal = find_neighbours(n, open, closed, pos)

        if found_goal: # If goal is found, path is printed
            print_path(found_goal)
            break

    return {"Goal Found": found_goal != False, "Search Algorithm": algo, "Memory Usage (Maximum States Stored)": max_mem, "Time taken (nodes visited)": len(closed)}

In [34]:
print(state_space_search("bfs"))

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Search Algorithm': 'bfs', 'Memory Usage (Maximum States Stored)': 1, 'Time taken (nodes visited)': 1}


In [35]:
print(state_space_search("dfs", 10))

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Search Algorithm': 'dfs', 'Memory Usage (Maximum States Stored)': 1, 'Time taken (nodes visited)': 1}


In [36]:
# State Space algorithm for ddfs or iddfs
def modified_state_space_search(algo = "ddfs", max_depth = 10):
    open = [start]
    closed = []
    pos = 0 # Inserting at 0 to use open as a stack
    depth_limit = 0
    found_goal = False

    max_mem = 0 # save maximum memory usage
    max_time = 0 # save number of nodes visited

    while len(open):
        max_mem = max(max_mem, len(open) + len(closed))
        n = open.pop(0)

        if algo == "iddfs" and len(open) == 0: # Increasing Depth in IDDFS when open becomes empty
            depth_limit += 1
            open = [start]
            max_time += len(closed)
            closed = []

        if algo == "iddfs" and n.depth > depth_limit: # Depth Check for IDDFS
            continue

        if n.depth > max_depth: # Max Depth for DDFS
            continue

        closed.append(n)
        found_goal = find_neighbours(n, open, closed, pos)

        if found_goal: # If goal is found, path is printed
            print_path(found_goal)
            break

    max_time += len(closed)
    return {"Goal Found": found_goal != False, "Search Algorithm": algo, "Memory Usage (maximum states stored)": max_mem, "Time Taken (nodes visited)": max_time}

In [37]:
print(modified_state_space_search("iddfs"))

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Search Algorithm': 'iddfs', 'Memory Usage (maximum states stored)': 1, 'Time Taken (nodes visited)': 1}


In [38]:
print(modified_state_space_search("ddfs", 5))

[[1 2 3]
 [4 5 6]
 [7 0 8]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Search Algorithm': 'ddfs', 'Memory Usage (maximum states stored)': 1, 'Time Taken (nodes visited)': 1}
